## This notebook does Retraining of the CFU Counting Model. It uses the latest images from the validated set of images. It compares the Model KPIs after retraining with the original model, and also checks if the counts are matching the baselined counts from Test images. Then uploads the retrained model with model_kpis.json updated to Azure Blob storage.

### Getting images and their validated jsons from CosmosDB

#### Importing all libraries

In [ ]:
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database
from urllib.parse import quote_plus
from datetime import datetime,timedelta

In [ ]:
%load_ext dotenv
%dotenv ../services/environments/.env.secrets

#### Establishing connection to Cosmos DB

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%load_ext snakeviz
import holoviews as hv
hv.extension('bokeh')
# Import standard libs
import docx
import logging
import os
import pickle
import pycocotools
import random
import regex
import sys
import shutil
from collections import OrderedDict
from copy import deepcopy
from itertools import accumulate
from operator import add
from pathlib import Path
from pprint import pprint
from typing import *
from uuid import uuid4


# import first party libs
import cv2
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
print(torch.__version__, torch.cuda.is_available())
from PIL import Image
from skimage.measure import label, regionprops, find_contours
from tqdm.auto import tqdm


# import detectron2 utilities
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.utils.logger import setup_logger

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode, Boxes
from detectron2.data import transforms as T
from detectron2.data import build_detection_train_loader, build_detection_test_loader, DatasetMapper
from detectron2.data import detection_utils as utils

from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.config.config import CfgNode
from detectron2.engine import hooks

from detectron2.utils import comm
from detectron2.structures import Instances
from detectron2.evaluation import COCOEvaluator, DatasetEvaluator, DatasetEvaluators
from detectron2.modeling import GeneralizedRCNNWithTTA

from detectron2.checkpoint import DetectionCheckpointer

from detectron2.evaluation import verify_results

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", 100)
setup_logger()
cv2.setNumThreads(0)

# Append codebase path
sys.path.append("../src/")
from vxgioq_cfu_counting_ml.api.data_models import Predictions, Region, ShapeAttributes

from vxgioq_cfu_counting_ml.training.dataset_mapper import CFUDatasetMapper
from vxgioq_cfu_counting_ml.training.data_analysis import get_dataset_stats, check_retest_models, compute_kpis, get_data_compute_predictions_save_zip_upload_images_names_predictions_count, construct_model_info, predict_images
from vxgioq_cfu_counting_ml.training.matched_data import MATCHES
from vxgioq_cfu_counting_ml.training.organize_data import get_paths_test_images, get_paths_train_images, count_files, get_paths_positive_unlabelled_images, via_to_detectron2, custom_train_test_split, generate_train_test, gsk_original_to_detectron2, negative_to_detectron2
from vxgioq_cfu_counting_ml.training.trainer import CFUTrainer

from vxgioq_cfu_counting_ml.utils.azure import (
download_image,
delete_object_from_blob,
download_load_models_kpis,
download_unzip_clean_zip,
get_cosmos_mongo_collection,
get_cosmos_mongo_collection_phase2,
get_file_from_blob,
list_data_on_blob,
list_models_on_blob,
upload_file_to_blob,
download_unzip_clean_data,
download_unzip_clean_model,
write_zip_upload_models_kpis,
zip_upload_data_to_blob,
zip_upload_file_folder_to_blob,
zip_upload_model_to_blob,
zip_upload_predictions_to_blob,
ACCOUNT_URL,
CONTAINER,
DATASET_DETECTRON2_NAME,
DATASETS_QUANTILES_NAME,
MODEL_CONFIG_NAME,
MODELS_KPIS_NAME,
MODELS_QUANTILES_NAME,
MODEL_WEIGHTS_NAME,
PATH_DATA_ON_DISK,
PATH_MODELS_ON_DISK,
PATH_PREDICTIONS_ON_DISK
)
from vxgioq_cfu_counting_ml.utils.conf import Backbone, Device, clean_output_dir_add_cfg, delete_files, delete_folders, prepare_cfu_detectron2_config_from_detectron2_model_zoo, prepeare_cfu_detectron_config_from_existing_model
from vxgioq_cfu_counting_ml.utils.prediction_intervals import pad_arrays, scores_to_distributions, generate_pi_dataset, DataType, train_quantile_regressor, HPSpace, verify_quantiles_predictions, BinningOption, DistributionFunctionType, scores_to_compiled_cdfs
from vxgioq_cfu_counting_ml.utils.convert import enhance_detectron2_dataset, extract_challenging_images, load_scan4000_excel,  extract_all_from_scan4000_report
from vxgioq_cfu_counting_ml.utils.custom_datasets import func_dataset, mask_2_polygon
from vxgioq_cfu_counting_ml.utils.display import compare_scan4000_and_dl_images, display_batch, display_pi, display_dataset, display_enhanced_dataset, display_features_importance, display_violin, NormalizeOption, save_examples
from vxgioq_cfu_counting_ml.utils.evaluator import CFUEvaluator
from vxgioq_cfu_counting_ml.utils.loader import load_model, get_best_model_name, get_best_model, load_images
from vxgioq_cfu_counting_ml.utils.metrics import mse, mae, mape, calibration
from vxgioq_cfu_counting_ml.utils.predictor import CFUPredictor
from vxgioq_cfu_counting_ml.utils.types import *

In [ ]:
import glob
from vxgioq_cfu_counting_ml.utils.azure import download_image
from PIL import Image

In [ ]:
import time
from datetime import datetime, timedelta, timezone
from typing import Any, Callable, Dict, List, Union
import sys
import arrow
from timeloop import Timeloop
import pandas as pd

In [ ]:
from vxgioq_cfu_counting_ml.api.data_models import (
    CFUInput,
    CFUOutput,
    ImageOutput,
    Predictions,
    Region,
    ShapeAttributes,
)
from vxgioq_cfu_counting_ml.utils import conf
from vxgioq_cfu_counting_ml.utils.azure import download_image, get_cosmos_mongo_collection,download_unzip_clean_model,get_blob_service_client
from vxgioq_cfu_counting_ml.utils.custom_datasets import mask_2_polygon
from vxgioq_cfu_counting_ml.utils.predictor import get_best_cfu_model, CFUPredictor
from vxgioq_cfu_counting_ml.utils.loader import get_best_model_name,download_unzip_best_model

In [ ]:
import json

import tempfile
from pickle import HIGHEST_PROTOCOL  # noqa: S403
from typing import Any, List, Optional, Sequence, Tuple, Union

import joblib
from detectron2.structures import BoxMode
from pydantic import ValidationError
from skimage.measure import find_contours, label
from sklearn.utils import _safe_indexing
from sklearn.utils.validation import indexable

sys.path.append("VXGIOQ-CFU-Counting-ML/src/")

from vxgioq_cfu_counting_ml.utils.types import (
    DatasetVIAFormat,
    Detectron2Annotation,
    Detectron2DatasetElement,
    VIACircle,
    VIAEllipse,
    VIAPolygon,
)

In [ ]:
dishes_collection2 = get_cosmos_mongo_collection_phase2()

#### User needs to enter the criteria for getting images, if last_month is set to True, this will get all images from last one month. In other case, User can enter Start date and the number of days to get images from the Start date + number of days. There is a max limit set to restrict the max number of images downloaded.

In [ ]:
last_month=False  #if we need images from the past 30 days
max_no_of_images =400   #the number of images selected
days_range = 30   #number of days, if last_month set to True, the range is curr Date - days_range
no_of_days = timedelta(days_range)

path_jsons = '../data/new_jsons/'
path_data = '../data'

In [ ]:
if(last_month):
    #selects the start date 30 days before the current date
    ending_date = datetime.now()
    starting_date = ending_date - no_of_days

else:
    #for a particular timespan
    starting_date = datetime(2021,10,1)  # year,month,day format
    ending_date = datetime(2022,1,15)
    #ending_date = starting_date + no_of_days

In [ ]:
print(starting_date)
print(ending_date)

In [ ]:
total_jsons = list(dishes_collection2.find({'predictions.image.predictions_datetime':{"$gte":starting_date, "$lte":ending_date}}))
print(len(total_jsons))
processed_images=len(total_jsons)

In [ ]:
jsons = list(dishes_collection2.find({ "$and": [ { "is_validated":1 }, { 'predictions.image.predictions_datetime': {"$gte":starting_date, "$lte":ending_date} } ] }).limit(max_no_of_images))
print(len(jsons))

In [ ]:
print(f'Taking {len(jsons)} images and their jsons between {starting_date} and {ending_date}...')

### Creating the folders for the images and the jsons

In [ ]:
if(not os.path.exists(path_data)):
    os.makedirs(path_data)
    print(f'Creating folder for data in {path_data}')

In [ ]:
path_imgs = os.path.join(path_data,'new_images')

In [ ]:
if(not os.path.exists(path_jsons)):
    os.makedirs(path_jsons)
    print(f'Creating folder for jsons in {path_jsons}')
else:
    shutil.rmtree(path_jsons)
    os.makedirs(path_jsons)
    print(f'Deleting old folder and creating new folder for jsons in {path_jsons}')

if(not os.path.exists(path_imgs)):
    print(f'Creating folder for images in {path_imgs}')
    os.makedirs(path_imgs)
else:
    shutil.rmtree(path_imgs)
    os.makedirs(path_imgs)
    print(f'Deleting old folder and creating new folder for images in {path_imgs}')

In [ ]:
for i in range(len(jsons)):
    image_url = jsons[i]['image_original_url']
    x = download_image(image_url)
    img = Image.fromarray(x)
    img_name  = jsons[i]['sample_id']+'.png'
    img.save(os.path.join(path_imgs,img_name))
    
    json_name = jsons[i]['sample_id']+'.json'
    with open(path_jsons+json_name,'w') as dw:
        jsons[i].pop("_id")
        jsons[i]["predictions"]['image']['predictions_datetime'] = jsons[i]["predictions"]['image']['predictions_datetime'].strftime('%Y-%m-%dT%H:%M:%S.%sZ')
        jsons[i]["scanned_datetime"] = jsons[i]["scanned_datetime"].strftime('%Y-%m-%dT%H:%M:%S.%sZ')
        json.dump(jsons[i],dw)

### Converting the validated jsons into VIA format json

In [ ]:
l = []

#path = 'image_002.json'
for json_name in glob.glob(os.path.join(path_jsons,'*')):
    l.append(json_name)

if os.path.join(path_jsons,'via_dataset_via.json') in l:
    l.remove(os.path.join(path_jsons,'via_dataset_via.json'))

d = dict()
for r,i in enumerate(l):
    with open(i) as x:
        f = json.load(x)
        d['image_'+str(r)] = {
                        "filename": f['sample_id']+'.png',
                        'size': f['corrections'][-1]['image']['size'],
                        'regions':[],
                        'file_attributes': f['corrections'][-1]['image']['file_attributes'],
        }
        
        for i in range(len(f['corrections'][-1]['image']['regions'])):  
            if 'category_id' in f['corrections'][-1]['image']['regions'][i]['shape_attributes'].keys():
                f['corrections'][-1]['image']['regions'][i]['shape_attributes'].pop("category_id")
            d['image_'+str(r)]['regions'].append(
                    {
                        'shape_attributes': f['corrections'][-1]['image']['regions'][i]['shape_attributes'],
                        'region_attributes': f['corrections'][-1]['image']['regions'][i]['region_attributes'],
                    })
with open(os.path.join(path_jsons,'via_dataset_via.json'),'w') as dw:
    json.dump(d,dw)

### Converting the VIA format json to detectron element

In [ ]:
path_datasets_via = os.path.join(path_jsons,'via_dataset_via.json')
path_data = path_imgs
path_dataset_detectron2 = os.path.join(path_imgs,'dataset_detectron2.lz4')

dataset_detectron2: List[Detectron2DatasetElement] = []
_dataset_via = json.load(open(path_datasets_via, "r"))

In [ ]:
for _dataset_via in json.load(open(path_datasets_via, "r")).values():

        new_file_name = os.path.join(path_data, os.path.basename(_dataset_via["filename"]))
        #print(new_file_name)
        height, width, _ = cv2.imread(new_file_name).shape
        #print(height,width)
        
        try:
            dataset_via = DatasetVIAFormat(**_dataset_via)
        except ValidationError as e:
            logger.error(
                f"Skipping annoted file: {_dataset_via['filename']} because it has unsupported shapes_attributes.\nFull stack:\n{e}"
            )
            continue
        annotations: List[Detectron2Annotation] = []
        if (
            len(dataset_via.regions) > 0
        ):  # need at least one instance to fill the annotations object
            for region in dataset_via.regions:

                # generate the mask based on the shape at hand
                # --------------------------------------------
                shape_attributes = region.shape_attributes
                mask = np.zeros((height, width), dtype=np.uint8)
                if isinstance(shape_attributes, (VIACircle, VIAEllipse)):
                    # extract parameters
                    cx = shape_attributes.cx
                    cy = shape_attributes.cy
                    if isinstance(shape_attributes, VIACircle):
                        rx = ry = shape_attributes.r
                        tetha = 0.0
                    else:
                        rx = shape_attributes.rx
                        ry = shape_attributes.ry
                        tetha = (
                            shape_attributes.theta
                        )  # If VIA starts dealing with rotated ellipses, ensure tetha is in degrees, as for cv2
                    # fill the mask
                    startAngle = 0
                    endAngle = 360
                    color = 255
                    thickness = -1
                    mask = cv2.ellipse(
                        mask, (cx, cy), (rx, ry), tetha, startAngle, endAngle, color, thickness
                    )
                elif isinstance(shape_attributes, VIAPolygon):
                    # extract parameters
                    all_points_x = shape_attributes.all_points_x
                    all_points_y = shape_attributes.all_points_y

                    # construct the mask
                    mask = cv2.fillPoly(
                        mask,
                        [np.array([[int(x), int(y)] for x, y in zip(all_points_x, all_points_y)])],
                        (255),
                    )

                # get contour
                contour: np.ndarray = find_contours(
                    mask.astype(np.bool),
                    level=0.0,
                    fully_connected="high",
                    positive_orientation="high",
                )[0]

                # get bbox
                x_min = np.min(contour[:, 1])
                x_max = np.max(contour[:, 1])
                y_min = np.min(contour[:, 0])
                y_max = np.max(contour[:, 0])
                bbox = [x_min, y_min, x_max, y_max]

                # get segmentation
                segmentation = [
                    contour[:, ::-1]  # go from row, colum (y, x) to x, y
                    .reshape(
                        -1
                    )  # go from np.array([[x1, y1], ..., [xn, yn]]) to np.array([x1, y1, ..., xn, yn])
                    .tolist(),  # The polygon components coordinates [x1, y1, x2, y2, ...] of the object (all objects can only be made of one polygon component in that dataset)
                ]

                # fill the annotation
                annotation: Detectron2Annotation = Detectron2Annotation(
                    bbox=bbox,
                    bbox_mode=BoxMode.XYXY_ABS,
                    category_id=0,  # only 1 class to identify for now (no species distinction)
                    segmentation=segmentation,
                )
                annotations.append(annotation)

        else:
            logger.debug(
                f"No instance found on for image: {new_file_name}. Make sure to set "
                "`DATALOADER.FILTER_EMPTY_ANNOTATIONS=False` if you want to include "
                "this (and other empty) example(s) in the training."
            )

        # convert VIA image annotation to detectron2 annotation
        dataset_detectron2_element: Detectron2DatasetElement = Detectron2DatasetElement(
            file_name=new_file_name,
            height=height,
            width=width,
            image_id=os.path.basename(new_file_name),
            annotations=annotations,
        )

        # append
        dataset_detectron2.append(dataset_detectron2_element)
        
joblib.dump(
        dataset_detectron2, filename=path_dataset_detectron2, compress=9, protocol=HIGHEST_PROTOCOL,
    )

### Download Models

In [ ]:
# download and load the models kpis
models_kpis = download_load_models_kpis(overwrite=False)

In [ ]:
# list and download all models
model_names = [model.name for model in models_kpis.models if model.available]


for model_blob in model_names:
    if model_blob != MODELS_KPIS_NAME:  # Don't download the KPIs, we will do that afterwards
        download_unzip_clean_model(model_name=model_blob, overwrite=False)

In [ ]:
print(model_names)

In [ ]:
for i in range (len(models_kpis.models)):
    print(models_kpis.models[i].name,',MAE:',models_kpis.models[i].kpis.main['MAE (counts) 6-10']) 

### Determine current best model

In [ ]:
current_best = download_unzip_best_model(task="main", metric="MAE (counts) 6-10")
print(current_best)

### Use Legacy images along with the Newly validated images to prepare Training data

In [ ]:
path_labelled = os.path.join('../data','labelled')
path_unlabelled = os.path.join('../data','unlabelled')

if(not os.path.exists(path_labelled)):
    os.makedirs(path_labelled)
    print(f'Creating folder for labelled in {path_labelled}')
path_organized = os.path.join('../data','organized')
if(not os.path.exists(path_unlabelled)):
    os.makedirs(path_unlabelled)
    print(f'Creating folder for unlabelled in {path_unlabelled}')

In [ ]:
gsk_original_to_detectron2(data_name="labelled", overwrite=True) #Update this to False, after doing first time
negative_to_detectron2(data_name="unlabelled", overwrite=True) #Update this to False, after doing first time

### Splitting into train and test folders

In [ ]:
path_organized = os.path.join('../data','organized')
if(not os.path.exists(path_organized)):
    os.makedirs(path_organized)
    print(f'Creating folder for train and test in {path_organized}')

In [ ]:
folders_positive = [
  os.path.join(PATH_DATA_ON_DISK, "labelled"),
    os.path.join(PATH_DATA_ON_DISK, "new_images"),
]

folders_negative = [
    os.path.join(PATH_DATA_ON_DISK, "unlabelled"),
]

path_organized = generate_train_test(
    folders=folders_positive,
   folders_negative=folders_negative,
    #overwrite=False
    overwrite=True
)

### Checking number of images present

In [ ]:
#print("Images in train folder: ",len(glob.glob('../data/organized/train/*.png')+glob.glob('../data/organized/train/*.bmp')))
#print("Images in test folder: ",len(glob.glob('../data/organized/test/*.png')+glob.glob('../data/organized/test/*.bmp')))

original_num_images = models_kpis.data.n_examples
print(f"\nOriginal Number of Images: {original_num_images}")
num_images = count_files(path_organized)
print(f"Currently Total Number of Images: {num_images}")
test_images = count_files('../data/organized/test')
train_images = count_files('../data/organized/train')
print(f"Currently Number of Train Images: {train_images}")
print(f"Currently Number of Test Images: {test_images}")

### Register the train and test datasets
Registration process is needed for detectron2 to be able to find the datasets

In [ ]:
dataset_catalog_name_root = func_dataset.__name__[5:]
print(dataset_catalog_name_root)
for dataset_type in ["train", "test"]:
    dataset_catalog_name = f"{dataset_catalog_name_root}_{dataset_type}"
    DatasetCatalog.register(dataset_catalog_name, lambda dataset_type=dataset_type: func_dataset(folder=os.path.join(path_organized, dataset_type)))
    MetadataCatalog.get(dataset_catalog_name).set(thing_classes=["CFU"])

In [ ]:
dataset_gsk_train_metadata = MetadataCatalog.get(f"{dataset_catalog_name_root}_train")
dataset_gsk_test_metadata = MetadataCatalog.get(f"{dataset_catalog_name_root}_test")
dataset_gsk_train_metadata

In [ ]:
dataset_gsk_test_metadata

### Recomputing the KPIs
Check if data folders or CFUEvaluator KPIs changed and if so, recompute the kpis on the updated test data

In [ ]:
models_kpis = check_retest_models(models_kpis=models_kpis, data_folders=folders_positive + folders_negative, n_examples=count_files(path_organized))

In [ ]:
models_kpis

In [ ]:
for i in range (len(models_kpis.models)):
    print(models_kpis.models[i].name,',MAE:',models_kpis.models[i].kpis.main['MAE (counts) 6-10']) 

# 2. Train Instance Segmentation model (DL, aka Deep Learning)

In [ ]:
# display trained model names
print(model_names)

## Load and modify the training config

#### If user wants to use an existing model, the best model present will be used

In [ ]:
existing_model_name = current_best
#existing_model_name = '5c352834-3af1-4018-bac5-cc5dad43e202'
print(existing_model_name)

In [ ]:
device = Device.GPU

In [ ]:
#Backbone.R50_FPN, Backbone.R101_FPN, Backbone.X101_FPN

In [ ]:
from_detectron2_model_zooList = [False] #Default is false
backboneList = [Backbone.R50_FPN] #default is R50_FPN
iterationList = [36000] #Default is 36000
lrList=[0.01] #Default is 0.01
ctr=0
cfgList=[]
trackData=[]
for model_zoo in from_detectron2_model_zooList:
    if(model_zoo):
        for backbone, iterations, lr in [(backbone, iterations, lr) 
         for backbone in backboneList for iterations in iterationList for lr in lrList]:
            cfg = prepare_cfu_detectron2_config_from_detectron2_model_zoo(backbone=backbone, device=device, iterations=iterations)
            cfgList.append(cfg)
            cfg.SOLVER.MAX_ITER = iterations
            cfg.SOLVER.BASE_LR = lr
            trackData.append([cfg.OUTPUT_DIR,model_zoo,iterations,lr])
            ctr+=1
    else: 
        for iterations, lr in [(iterations, lr) 
         for iterations in iterationList for lr in lrList]:
            cfg = prepeare_cfu_detectron_config_from_existing_model(existing_model_name=existing_model_name, iterations=iterations)
            cfgList.append(cfg)
            cfg.SOLVER.MAX_ITER = iterations
            cfg.SOLVER.BASE_LR = lr
            trackData.append([cfg.OUTPUT_DIR,model_zoo,iterations,lr])
            ctr+=1           
                 
print(ctr)

In [ ]:
for cfg in cfgList:
    print(cfg.OUTPUT_DIR)

## Instantiate trainer

In [ ]:
trainerList=[]

In [ ]:
for cfg in cfgList:
    trainer = CFUTrainer(cfg)

    trainer.resume_or_load(resume=False)
    if cfg.TEST.AUG.ENABLED:
        trainer.register_hooks(
            [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
        )
    trainerList.append(trainer)

In [ ]:
for trainer in trainerList:
    print(trainer)

## Start Training

In [ ]:
for i in range(len(cfgList)):
    trainerList[i].train()
    cfgList[i].MODEL.WEIGHTS = os.path.join(cfgList[i].OUTPUT_DIR, MODEL_WEIGHTS_NAME)

## Assess trained model

In [ ]:
model_infoList = []

In [ ]:
# Construct model info (name, datetime, ..., KPIs)
for cfg in cfgList:
    model_infos = construct_model_info(cfg=cfg)
    model_infoList.append(model_infos)

In [ ]:
print(len(model_infoList))

In [ ]:
model_infoList

In [ ]:
for i in range (len(model_infoList)):
    print(model_infoList[i].name,model_infoList[i].kpis.main['MAE (counts) 6-10'])

In [ ]:
trackData

In [ ]:
path_doc = '../retraining_doc'
if(not os.path.exists(path_doc)):
    os.makedirs(path_doc)
    print(f'Creating folder for csv file in {path_doc}')

In [ ]:
for i in range(len(trackData)):
    for j in range(len(model_infoList)):
        if(trackData[i][0].split('/')[-1]==model_infoList[j].name):
            trackData[i].append(model_infoList[j].backbone)
            trackData[i].append(model_infoList[j].kpis.main['MAE (counts) 6-10'])
            trackData[i].append(model_infoList[j].datetime_finished)
            trackData[i].append(model_infoList[j].kpis)

In [ ]:
trackData

In [ ]:
trackdf = pd.DataFrame(trackData,columns=['Output Dir','From_detectron2_model_zoo','Iterations','Learning Rate','Backbone','MAE (counts) 6-10','Datetime Finished','Model Kpis'])
trackdf

In [ ]:
download_unzip_clean_zip("cfu_counting_hyperparameters.csv.zip",path_doc,f'{ACCOUNT_URL}/machine-learning/monitoring',overwrite=True)
trackdf.to_csv(os.path.join(path_doc,'cfu_counting_hyperparameters.csv'),mode='a',index=False,header=False)

In [ ]:
zip_upload_file_folder_to_blob(os.path.join(path_doc,'cfu_counting_hyperparameters.csv'),url=f'{ACCOUNT_URL}/machine-learning/monitoring/cfu_counting_hyperparameters.csv.zip',overwrite=True)

In [ ]:
if(len(model_infoList)<=1):
    model_info = model_infoList[0]
else:
    min_kpis = model_infoList[0].kpis.main['MAE (counts) 6-10']
    model_info = model_infoList[0]
    cfg = cfgList[0]
    for i in range(len(model_infoList)):
        if model_infoList[i].kpis.main['MAE (counts) 6-10']<min_kpis:
            min_kpis=  model_infoList[i].kpis.main['MAE (counts) 6-10']
            model_info = model_infoList[i]
            cfg = cfgList[i]
    print(min_kpis)

In [ ]:
#model_info
model_info.kpis.main['MAE (counts) 6-10']

In [ ]:
for i in cfgList:
    print(i.OUTPUT_DIR)

In [ ]:
cfg.OUTPUT_DIR

In [ ]:
cfg.SOLVER.MAX_ITER 

In [ ]:
cfg.SOLVER.BASE_LR

In [ ]:
model_info.name

In [ ]:
# write the config next to the final model and events
model_config_path = os.path.join(cfg.OUTPUT_DIR, MODEL_CONFIG_NAME)
logger.info(f"Writing the model config to: {model_config_path}.")
with open(model_config_path, "w") as f:
    f.write(cfg.dump())

In [ ]:
model_info

In [ ]:
no_iter = model_info.iteration
print(no_iter)

# 3. Train Quantiles model (LightGBM)

In [ ]:
n_bins = 100
n_exples_train = 10000 #10000 is default
n_exples_test = 1000 #1000 is default

In [ ]:
X_train, y_train = generate_pi_dataset(model_name=model_info.name, data_type=DataType.TRAIN, n_examples=n_exples_train, n_bins=n_bins,augmentations=CFUTrainer.augmentations_test)
X_test, y_test = generate_pi_dataset(model_name=model_info.name, data_type=DataType.TEST, n_examples=n_exples_test, dataset_name=dataset_gsk_test_metadata.name, n_bins=n_bins,augmentations=CFUTrainer.augmentations_test)

## Start training

In [ ]:
n_hp_points = 32768 #32768 is default

In [ ]:
hp_space_size = HPSpace.WIDE
quantiles = [0.025, 0.975]

EPS = 1e-10
regs = [
    ModelQuantile(
        model=train_quantile_regressor(X_train=X_train, y_train=y_train, quantile=quantile, n_hp_points=n_hp_points, hp_space_size=hp_space_size, train_size=0.7, cv=3),
        name=f"Quantile model {quantile * 100:.1f}th percentile",
        log_y=False,
        eps=EPS,
        n_bins=n_bins,
        column_names=list(X_train.columns),
        quantile=Quantile(expected=quantile)
    )
    for quantile in quantiles
]

### Updating actual quantiles

In [ ]:
y_train_pred_low = regs[0].predict(X_train)
y_test_pred_low = regs[0].predict(X_test)
y_train_pred_high = regs[1].predict(X_train)
y_test_pred_high = regs[1].predict(X_test)

regs[0].quantile.actual_train = (y_train < y_train_pred_low).sum() / len(y_train)
regs[0].quantile.actual_test = (y_test < y_test_pred_low).sum() / len(y_test)

regs[1].quantile.actual_train = (y_train < y_train_pred_high).sum() / len(y_train)
regs[1].quantile.actual_test = (y_test < y_test_pred_high).sum() / len(y_test)

expected_interval = quantiles[1] - quantiles[0]
actual_interval_train = regs[1].quantile.actual_train - regs[0].quantile.actual_train
actual_interval_test = regs[1].quantile.actual_test - regs[0].quantile.actual_test

model_quantiles_interval = ModelQuantilesInterval(
    low=regs[0],
    high=regs[1],
    expected_interval=expected_interval,
    actual_interval_train=actual_interval_train,
    actual_interval_test=actual_interval_test,
    calibration_train=calibration(pi_expected=expected_interval, pi_actual=actual_interval_train),
    calibration_test=calibration(pi_expected=expected_interval, pi_actual=actual_interval_test),
    sharpness_train=Sharpness(
        mae=mae(y_train_pred_high.to_numpy(), y_train_pred_low.to_numpy()),
        mse=mse(y_train_pred_high.to_numpy(), y_train_pred_low.to_numpy())
    ),
    sharpness_test=Sharpness(
        mae=mae(y_test_pred_high.to_numpy(), y_test_pred_low.to_numpy()),
        mse=mse(y_test_pred_high.to_numpy(), y_test_pred_low.to_numpy())
    ),
)


### Persisting quantile models and data in the corresponding Instance Segmentation model folder

In [ ]:
joblib.dump(model_quantiles_interval, filename=os.path.join(cfg.OUTPUT_DIR, MODELS_QUANTILES_NAME), compress=9, protocol=pickle.HIGHEST_PROTOCOL)

dataset_pi = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
}
joblib.dump(dataset_pi, filename=os.path.join(cfg.OUTPUT_DIR, DATASETS_QUANTILES_NAME), compress=9, protocol=pickle.HIGHEST_PROTOCOL)

# update the model_info with the quantiles
model_info.quantiles = [model_quantiles_interval.low.quantile, model_quantiles_interval.high.quantile]
model_info.quantiles_calibration_test = model_quantiles_interval.calibration_test
model_info.quantiles_sharpness_test = model_quantiles_interval.sharpness_test


In [ ]:
model_info

In [ ]:
model_info.name

# 3a. Run Test case to validate new model 

In [ ]:
def list_sample_files_on_blob(name_starts_with: str, return_urls: bool = False):
    """
    List the file names starting with `name_starts_with` on Azure blob storage. Parameters
    ----------
    name_starts_with
    List the files which name starts with
    return_urls
    Whether to return the file name, or its complete url.
    """
    # create container client
    blob_service_client = get_blob_service_client()
    container_client = blob_service_client.get_container_client("sample")
    # list blobs
    blob_list = container_client.list_blobs(name_starts_with=name_starts_with)
    blob_folder = container_client.walk_blobs(name_starts_with=name_starts_with)
    folder_name = [blob.name.split("/")[1] for blob in blob_folder]
    # return the blob names
    if return_urls:
        return [blob.name for blob in blob_list]
    else:
        return [blob.name.split("/")[-1] for blob in blob_list], folder_name

#### Keeping the original model and the newly trained model in list

In [ ]:
#place original and retrained model names
best_model_names = [existing_model_name, model_info.name]

In [ ]:
best_model_names[1]

In [ ]:
PERIODIC_MODELS_UPDATES = 600  # periodic model updates, in seconds
PREDICTOR = [CFUPredictor(model_name=best_model_name) for best_model_name in best_model_names]

In [ ]:
blob_list = list_sample_files_on_blob(name_starts_with="test_images/", return_urls=True)
blob_list = [blob.split("/")[-1] for blob in blob_list]
#blob_list = blob_list[:10]
print(blob_list)

In [ ]:
def segment_images_instances(cfu_input: CFUInput, test_images, predict) -> CFUOutput:
    """
    Predict CFUs on images.

    The endpoint
        - receives the dishes document ids of the azure cosmsos mongo db collection
        - downloads the images corresponding to those documents
        - predicts the cfus on these images
        - updates the cosmosdb documents with the corresponding predictions
        - returns an object listing the predictions and additional information.
    """
    
    # load images from blob
    logger.info("Load images in RGB...")
    
    images_inputs = [download_image(f"{ACCOUNT_URL}/sample/test_images/{image}") for image in test_images]
    heights = [image_input.shape[0] for image_input in images_inputs]
    widths = [image_input.shape[1] for image_input in images_inputs]

    # predict (not useful to batch predict as pytorch model doesn't batch images)
    logger.info("Predict images...")
    predictions = [
        predict.predict_quantiles([image_input[:, :, ::-1]])[0] for image_input in images_inputs
    ]
    predictions_datetime = datetime.now(tz=timezone.utc)

    # write output json
    logger.info("Prepare response...")
    max_edges = 101
    response = CFUOutput(
        images=[
            ImageOutput(
                sample_id=image_name,
                image_original_url=f"{ACCOUNT_URL}/sample/test_images/{image_name}",
                image_predicted_url=f"{ACCOUNT_URL}/sample/test_images/{image_name}",
                predictions=Predictions(
                    filename=f"{ACCOUNT_URL}/sample/test_images/{image_name}",
                    model_name=predict.model_name,
                    size=height * width,
                    regions=[
                        Region(
                            shape_attributes=ShapeAttributes(
                                all_points_x=mask_2_polygon(pred_mask, max_edges=max_edges)[
                                    :, 1
                                ].tolist(),
                                all_points_y=(
                                    mask_2_polygon(pred_mask, max_edges=max_edges)[:, 0]
                                ).tolist(),
                                name="polygon",
                            ),
                            region_attributes={},
                        )
                        for pred_mask in prediction.instances.pred_masks.numpy()
                    ],
                    predictions_cfu_count=prediction.model_prediction.prediction,
                    predictions_cfu_count_quantiles=prediction.quantiles_predictions,
                    predictions_scores=prediction.instances.scores.tolist(),
                    predictions_bboxes=prediction.instances.pred_boxes.tensor.tolist(),
                    predictions_datetime=predictions_datetime,
                ),
            )
            for image_name, prediction, height, width in zip(test_images, predictions, heights, widths)
        ]
    )
    
    
    # return json
    return response


In [ ]:
predicted_val = []
for predict in PREDICTOR:
    response = segment_images_instances(CFUInput, blob_list, predict)
    sample_id , model_name, cfu_count = [],[],[]
    for image in response:
        for data in image[1]:
            sample_id.append(data.sample_id)
            model_name.append(data.predictions.model_name)
            cfu_count.append(data.predictions.predictions_cfu_count)
        df= [sample_id, model_name, cfu_count]
        predicted_val.append(df)

In [ ]:
phase1_cfu_count = predicted_val[0][2]

In [ ]:
for idx, name, count in predicted_val:
    df = pd.DataFrame(list(zip(sample_id, phase1_cfu_count, count)),columns =['Image_name', 'Phase1_CFU_COUNT', 'Retrained_CFU_COUNT'])

In [ ]:
val = []
model_valid = True
for score in list(df["Phase1_CFU_COUNT"]- df["Retrained_CFU_COUNT"]):
    if score == 0:
        val.append("PASS")
    else:
        val.append("FAIL")
        model_valid = False
df["STATUS"] = val

In [ ]:
df

In [ ]:
model_valid

In [ ]:
print(starting_date,ending_date)

In [ ]:
path_doc = '../retraining_doc'
if(not os.path.exists(path_doc)):
    os.makedirs(path_doc)
    print(f'Creating folder for retraining doc file in {path_doc}')

In [ ]:
doc = docx.Document()
title = doc.add_heading(f"Validation of new model by checking on test cases",2)
title.alignment = 1
doc.add_paragraph('-'*118)

start = starting_date.strftime('%Y-%m-%d')
end = ending_date.strftime('%Y-%m-%d') 

doc.add_heading(f"Old Model: {best_model_names[0]}",3)
doc.add_heading(f"New Model: {best_model_names[1]}",3)
doc.add_paragraph()

doc.add_heading(f"Original number of images: {original_num_images}",3)
doc.add_heading(f"New number of images added: {len(jsons)}",3)
doc.add_paragraph(f"These {len(jsons)} images are between {start} and {end}")
doc.add_heading(f"Total number of images used for retraining the model: {num_images}",3)
doc.add_heading(f"Number of train images used: {train_images}",3)
doc.add_heading(f"Number of test images used: {test_images}",3)
doc.add_heading(f"Number of iterations: {no_iter}",3)

doc.add_paragraph()


table = doc.add_table(rows=df.shape[0]+1,cols=df.shape[1],style='Light Grid Accent 2')
for j in range(df.shape[-1]):
    table.cell(0,j).text = df.columns[j]


for i in range(df.shape[0]):
    for j in range(df.shape[-1]):
        table.cell(i+1,j).text = str(df.values[i,j])
doc.save(os.path.join(path_doc,"Retrained Model comparison.docx"))

# 4. Persist Instance Segmentation and Quantiles Models

In [ ]:
for i in range(len(models_kpis.models)):
    if best_model_names[0] in models_kpis.models[i].name:
        existing_model_kpi = models_kpis.models[i].kpis.main['MAE (counts) 6-10']
print(existing_model_kpi)

In [ ]:
kpis_check = True
if(model_info.kpis.main['MAE (counts) 6-10']<existing_model_kpi):
    kpis_check = True
else:
    kpis_check = False

In [ ]:
kpis_check

In [ ]:
# check if data folders or CFUEvaluator KPIs changed and if so, recompute the kpis on the updated test data and update the models_kpis

if model_valid and kpis_check:
    models_kpis = check_retest_models(models_kpis=models_kpis, data_folders=folders_positive + folders_negative, n_examples=count_files(path_organized))
    
    # write the updated models kpis to disk and upload to blob    
    write_zip_upload_models_kpis(models_kpis=models_kpis)

        
    # reload models_kpis in case another parallel training updated the available models in the meantime
    models_kpis = download_load_models_kpis()

    # Add the instance segmentation model info to model_kpis tracking
    models_kpis.models.append(model_info)

    # persist the updated model+kpis tracking
    write_zip_upload_models_kpis(models_kpis=models_kpis)
    
    
        # clean output dir and add cfg
    clean_output_dir_add_cfg(cfg=cfg)

    # zip upload that model folder and its content to blob
    zip_upload_model_to_blob(cfg.OUTPUT_DIR)

In [ ]:
availableModels = [model.name for model in models_kpis.models]
print(availableModels)
if model_info.name not in availableModels:
    models_kpis.models.append(model_info)
availableModels = [model.name for model in models_kpis.models]
print(availableModels)

In [ ]:
doc.add_paragraph()
title2 = doc.add_heading(f"Comparison of Model KPIs (based on {test_images} test images)",2)
title2.alignment = 1
doc.add_paragraph('_'*105)
    
num_rows = 5
num_columns = len(best_model_names)+1
table = doc.add_table(rows=num_rows,cols=num_columns,style='Light Grid Accent 2')
table.cell(0,0).text = 'MODEL NAME'
table.cell(1,0).text = 'MAE (counts) 0'
table.cell(2,0).text = 'MAE (counts) 1-5'
table.cell(3,0).text = 'MAE (counts) 6-10'
table.cell(4,0).text = 'MAPE (% counts) 11+'
     
table.cell(0,1).text = "Old Model- "+best_model_names[0]
table.cell(0,2).text = "New Model- "+best_model_names[1]
    
modelData={}
for i in range(len(models_kpis.models)):
    if models_kpis.models[i].name in best_model_names:
        modelData[models_kpis.models[i].name] = models_kpis.models[i].kpis.main
i=0
for j in range(num_columns-1):
    table.cell(1,j+1).text = str(round(modelData[best_model_names[i]]['MAE (counts) 0'],2))
    table.cell(2,j+1).text = str(round(modelData[best_model_names[i]]['MAE (counts) 1-5'],2))
    table.cell(3,j+1).text = str(round(modelData[best_model_names[i]]['MAE (counts) 6-10'],2))
    table.cell(4,j+1).text = str(round(modelData[best_model_names[i]]['MAPE (% counts) 11+'],2))
    i=i+1
doc.add_paragraph()
doc.add_paragraph("MAE (counts) 0 - Mean Absolute Error for dishes with 0 CFU")
doc.add_paragraph("MAE (counts) 1-5 - Mean Absolute Error for dishes with 1 to 5 CFUs")
doc.add_paragraph("MAE (counts) 6-10 - Mean Absolute Error for dishes with 6 to 10 CFUs")
doc.add_paragraph("MAPE (% counts) 11+ - Mean Absolute Percentage Error for dishes with 11+ CFUs")
if(model_valid and kpis_check):
    doc.add_paragraph()
    doc.add_paragraph("New Model Uploaded: True")
    doc.add_paragraph("The retrained model passed all the checks.")
    doc.save(os.path.join(path_doc,"Retrained Model comparison.docx"))
else:
    doc.add_paragraph()
    doc.add_paragraph("New Model Uploaded: False")
    doc.add_paragraph("The retrained model did not pass the validation conditions so it is not considered.")
    doc.save(os.path.join(path_doc,"Retrained Model comparison.docx"))

In [ ]:
model_info.kpis.main['MAE (counts) 6-10']

## Sending the Report as Word doc attachment through email

In [ ]:
import smtplib
from email.message import EmailMessage
from email.mime.base import MIMEBase 

In mailto we keep the mail id we want to send the email to and in mailfrom we keep the email id through which mail will be sent.

In [ ]:
mailto="ritika.x.koul@gsk.com"
#mailto=["ritika.x.koul@gsk.com","anirban.x.konar@gsk.com"]
mailfrom="ritika.x.koul@gsk.com"

In [ ]:
message="CFU counting model retraining job has completed."
message+="You can find the results attached here."

mailsubject="CFU Counting model retraining job completed."
msg=EmailMessage()
msg.set_content(message)
msg['Subject'] = mailsubject
msg['From'] = mailfrom
msg['To'] = mailto

with open(os.path.join(path_doc,'Retrained Model comparison.docx'), 'rb') as fp:
    data = fp.read()
    ctype = 'application/octet-stream'
    maintype, subtype = ctype.split('/', 1)
    msg.add_attachment(data, maintype=maintype, subtype=subtype, filename='Retrained Model comparison.docx')


server=smtplib.SMTP('localhost')
server.send_message( msg )
server.quit()